In [1]:
import os
import h5py
import math
import torch
import pickle
import numpy as np
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler
from lib.dataloader import normalize_data

In [3]:
data = h5py.File('NYCBike1.h5', 'r')
print(f"Data Keys: {data.keys()}")
data_tensor = torch.tensor(data['data'])
print(f"Data Tensor Shape: {data_tensor.shape}")
tensor = torch.reshape(data_tensor, (data_tensor.shape[0], data_tensor.shape[1], 128))

Data Keys: <KeysViewHDF5 ['data', 'date']>
Data Tensor Shape: torch.Size([4392, 2, 16, 8])


C:\Users\25540\AppData\Local\Temp\ipykernel_10168\1030900333.py:3: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\utils\tensor_new.cpp:233.)
  data_tensor = torch.tensor(data['data'])


In [4]:
x_tensor = None
y_tensor = None
for i in range(tensor.size(0)-77):
    window_list = [
        tensor[i:i+5],
        tensor[i+24:i+29],
        tensor[i+48:i+53],
        tensor[i+72:i+76]
    ]
    if x_tensor is None:
        x_tensor = torch.cat(window_list, dim=0)
        x_tensor = x_tensor.unsqueeze(0)
    else:
        window_list = torch.cat(window_list, dim=0).unsqueeze(0)
        x_tensor = torch.cat((x_tensor, window_list), dim=0)

    if y_tensor is None:
        y_tensor = tensor[i+77].unsqueeze(0)
    else:
        y_tensor = torch.cat((y_tensor, tensor[i+77].unsqueeze(0)), dim=0)

x_tensor = x_tensor.permute(0, 1, 3, 2)
y_tensor = y_tensor.permute(0, 2, 1).unsqueeze(1)
print(f"X Tensor Shape: {x_tensor.shape}")
print(f"Y Tensor Shape: {y_tensor.shape}")

X Tensor Shape: torch.Size([4315, 19, 128, 2])
Y Tensor Shape: torch.Size([4315, 1, 128, 2])


In [5]:
def dayhour_to_timelabel(day, hour):
    if day < 5: # workday
        time_label = hour
    else: # holiday
        time_label = hour + 24
    return time_label

In [6]:
def move_forward_1h (hour, day):
    hour += 1
    if hour == 24:
        day += 1
        hour = 0
        if day == 7:
            day = 0
    return hour, day

In [7]:
day=4
hour=4

In [8]:
time_label = torch.zeros(4315)
for i in range(time_label.size(0)):
    time_label[i] = dayhour_to_timelabel(day, hour)
    hour, day = move_forward_1h(hour, day)

In [9]:
max_CP_inflow = {}
max_CP_outflow = {}
c = torch.empty_like(y_tensor)

for i in range(y_tensor.size(0)):
    for node in range(y_tensor.size(2)):
        inflow = y_tensor[i][0][node][0]
        outflow = y_tensor[i][0][node][1]
        if node not in max_CP_inflow:
            max_CP_inflow[node] = inflow
        if node not in max_CP_outflow:
            max_CP_outflow[node] = outflow
        if inflow > max_CP_inflow[node]:
            max_CP_inflow[node] = inflow
        if outflow > max_CP_outflow[node]:
            max_CP_outflow[node] = outflow
        if max_CP_inflow[node] == 0:
            c[i][0][node][0] = 0
        else:
            c[i][0][node][0] = math.ceil(5 * inflow / max_CP_inflow[node])
        if max_CP_outflow[node] == 0:
            c[i][0][node][1] = 0
        else:
            c[i][0][node][1] = math.ceil(5 * outflow / max_CP_outflow[node])

print(f"C Tensor Shape: {c.shape}")

C Tensor Shape: torch.Size([4315, 1, 128, 2])


In [10]:
# create training, validation, and test sets for none OOD cases
def create_train_val_test_sets (x_tensor, y_tensor, c, time_label):
    # shuffle the data
    torch.manual_seed(0)
    indices = torch.randperm(x_tensor.size(0))
    x_tensor = x_tensor[indices]
    y_tensor = y_tensor[indices]
    c = c[indices]
    time_label = time_label[indices]

    # split the data
    train_size = int(0.7 * x_tensor.size(0))
    val_size = int(0.2 * x_tensor.size(0))

    x_train = x_tensor[:train_size]
    y_train = y_tensor[:train_size]
    c_train = c[:train_size]
    time_label_train = time_label[:train_size]

    x_val = x_tensor[train_size:train_size+val_size]
    y_val = y_tensor[train_size:train_size+val_size]
    c_val = c[train_size:train_size+val_size]
    time_label_val = time_label[train_size:train_size+val_size]

    x_test = x_tensor[train_size+val_size:]
    y_test = y_tensor[train_size+val_size:]
    c_test = c[train_size+val_size:]
    time_label_test = time_label[train_size+val_size:]

    return x_train, y_train, c_train, time_label_train, x_val, y_val, c_val, time_label_val, x_test, y_test, c_test, time_label_test

x_train, y_train, c_train, time_label_train, x_val, y_val, c_val, time_label_val, x_test, y_test, c_test, time_label_test = create_train_val_test_sets(x_tensor, y_tensor, c, time_label)
print(f"train shape: {x_train.shape}, {y_train.shape}, {c_train.shape}, {time_label_train.shape}")
print(f"val shape: {x_val.shape}, {y_val.shape}, {c_val.shape}, {time_label_val.shape}")
print(f"test shape: {x_test.shape}, {y_test.shape}, {c_test.shape}, {time_label_test.shape}")

# np.savez('train.npz', x=x_train, y=y_train, time_label=time_label_train, c=c_train)
# np.savez('val.npz', x=x_val, y=y_val, time_label=time_label_val, c=c_val)
# np.savez('test.npz', x=x_test, y=y_test, time_label=time_label_test, c=c_test)

train shape: torch.Size([3020, 19, 128, 2]), torch.Size([3020, 1, 128, 2]), torch.Size([3020, 1, 128, 2]), torch.Size([3020])
val shape: torch.Size([863, 19, 128, 2]), torch.Size([863, 1, 128, 2]), torch.Size([863, 1, 128, 2]), torch.Size([863])
test shape: torch.Size([432, 19, 128, 2]), torch.Size([432, 1, 128, 2]), torch.Size([432, 1, 128, 2]), torch.Size([432])


In [ ]:
def create_train_val_test_sets (tensor):
    # shuffle the data
    torch.manual_seed(0)
    indices = torch.randperm(x_tensor.size(0))
    tensor = tensor[indices]

    # split the data
    train_size = int(0.7 * tensor.size(0))
    val_size = int(0.2 * tensor.size(0))

    tensor_train = tensor[:train_size]

    tensor_val = tensor[train_size:train_size+val_size]

    tensor_test = tensor[train_size+val_size:]

    return tensor_train, tensor_val, tensor_test

In [11]:
spatial = tensor.permute(2, 0, 1).numpy()
scaler = StandardScaler()
spatial = scaler.fit_transform(spatial.reshape(-1, spatial.shape[-1])).reshape(spatial.shape)
print(f"Spatial Shape: {spatial.shape}")
spatial_mean = np.mean(spatial, axis=1)
print(f"Spatial mean Shape: {spatial_mean.shape}")
spatial_median = np.median(spatial, axis=1)
print(f"Spatial median Shape: {spatial_median.shape}")
spatial = np.concatenate((spatial_mean, spatial_median), axis=1)
print(f"Spatial Shape: {spatial.shape}")

Spatial Shape: (128, 4392, 2)
Spatial mean Shape: (128, 2)
Spatial median Shape: (128, 2)
Spatial Shape: (128, 4)


In [12]:
n_clusters= range(2, 6)
best_k = None
best_score = -1
best_cluster = None
for n in n_clusters:
    cluster = KMeans(n_clusters=n, init='k-means++', max_iter=300, n_init=10, random_state=0)
    cluster_labels = cluster.fit_predict(spatial)

    silhouette_avg = silhouette_score(spatial, cluster_labels)

    print(f"For k={n}, the average silhouette score is: {silhouette_avg}")

    if silhouette_avg > best_score:
        best_score = silhouette_avg
        best_k = n
        best_cluster = cluster

print(f"Best K: {best_k}")  

Exception in thread Thread-7:
Traceback (most recent call last):
  File "d:\Anaconda\anaconda3\envs\dgl\lib\threading.py", line 980, in _bootstrap_inner
    self.run()
  File "d:\Anaconda\anaconda3\envs\dgl\lib\site-packages\ipykernel\ipkernel.py", line 761, in run_closure
    _threading_Thread_run(self)
  File "d:\Anaconda\anaconda3\envs\dgl\lib\threading.py", line 917, in run
    self._target(*self._args, **self._kwargs)
  File "d:\Anaconda\anaconda3\envs\dgl\lib\subprocess.py", line 1495, in _readerthread
    buffer.append(fh.read())
  File "d:\Anaconda\anaconda3\envs\dgl\lib\codecs.py", line 322, in decode
    (result, consumed) = self._buffer_decode(data, self.errors, final)
UnicodeDecodeError: 'utf-8' codec can't decode byte 0xc8 in position 26: invalid continuation byte
d:\Anaconda\anaconda3\envs\dgl\lib\site-packages\joblib\externals\loky\backend\context.py:136: UserWarning: Could not find the number of physical cores for the following reason:
'NoneType' object has no attribute

For k=2, the average silhouette score is: 0.7131382195826163
For k=3, the average silhouette score is: 0.7150724273057647
For k=4, the average silhouette score is: 0.6793965150110867
For k=5, the average silhouette score is: 0.6487641306319236
Best K: 3


In [13]:
cluster_labels = best_cluster.fit_predict(spatial)
np.save("cluster_labels.npy", cluster_labels)
cluster_labels

array([0, 0, 0, 2, 2, 0, 0, 0, 0, 0, 2, 2, 2, 2, 0, 0, 0, 0, 2, 1, 2, 2,
       0, 0, 0, 2, 2, 1, 1, 2, 0, 0, 0, 1, 1, 2, 2, 2, 0, 0, 0, 1, 1, 1,
       1, 0, 0, 0, 0, 1, 1, 1, 2, 0, 0, 0, 0, 2, 1, 1, 2, 0, 0, 0, 0, 2,
       1, 1, 2, 0, 0, 0, 1, 2, 2, 2, 0, 0, 0, 0, 2, 2, 2, 2, 0, 0, 0, 0,
       2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [20]:
x_test_c0 = torch.tensor([])
x_test_c1 = torch.tensor([])
x_test_c2 = torch.tensor([])
y_test_c0 = torch.tensor([])
y_test_c1 = torch.tensor([])
y_test_c2 = torch.tensor([])
temp_x = x_test.permute(2, 0, 1, 3)
temp_y = y_test.permute(2, 0, 1, 3)
for i in range(cluster_labels.shape[0]):
    label = cluster_labels[i]
    if label == 0:
        x_test_c0 = torch.cat((x_test_c0, temp_x[i].unsqueeze(0)), dim=0)
        y_test_c0 = torch.cat((y_test_c0, temp_y[i].unsqueeze(0)), dim=0)
    elif label == 1:
        x_test_c1 = torch.cat((x_test_c1, temp_x[i].unsqueeze(0)), dim=0)
        y_test_c1 = torch.cat((y_test_c1, temp_y[i].unsqueeze(0)), dim=0)
    else:
        x_test_c2 = torch.cat((x_test_c2, temp_x[i].unsqueeze(0)), dim=0)
        y_test_c2 = torch.cat((y_test_c2, temp_y[i].unsqueeze(0)), dim=0)

x_test_c0 = x_test_c0.permute(1, 2, 0, 3)
x_test_c1 = x_test_c1.permute(1, 2, 0, 3)
x_test_c2 = x_test_c2.permute(1, 2, 0, 3)
y_test_c0 = y_test_c0.permute(1, 2, 0, 3)
y_test_c1 = y_test_c1.permute(1, 2, 0, 3)
y_test_c2 = y_test_c2.permute(1, 2, 0, 3)

print(f"x_test_c0 shape: {x_test_c0.shape}, y_test_c0 shape: {y_test_c0.shape}")
print(f"x_test_c1 shape: {x_test_c1.shape}, y_test_c1 shape: {y_test_c1.shape}")
print(f"x_test_c2 shape: {x_test_c2.shape}, y_test_c2 shape: {y_test_c2.shape}")

x_test_c0 shape: torch.Size([432, 19, 81, 2]), y_test_c0 shape: torch.Size([432, 1, 81, 2])
x_test_c1 shape: torch.Size([432, 19, 17, 2]), y_test_c1 shape: torch.Size([432, 1, 17, 2])
x_test_c2 shape: torch.Size([432, 19, 30, 2]), y_test_c2 shape: torch.Size([432, 1, 30, 2])


In [14]:
day_list = [0, 1, 2, 3, 4, 5, 6]
hour_list = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23]
def split_work_holiday (tensor, day, hour): # tensor of size (4315, 19, 2, 128)
    workday_tensor = None
    holiday_tensor = None
    for i in range(tensor.size(0)):
        if day <= 4:
            if workday_tensor is None:
                workday_tensor = tensor[i].unsqueeze(0)
                hour, day = move_forward_1h(hour, day)
            else:
                workday_tensor = torch.cat((workday_tensor, tensor[i].unsqueeze(0)), dim=0)
                hour, day = move_forward_1h(hour, day)
        else:
            if holiday_tensor is None:
                holiday_tensor = tensor[i].unsqueeze(0)
                hour, day = move_forward_1h(hour, day)
            else:
                holiday_tensor = torch.cat((holiday_tensor, tensor[i].unsqueeze(0)), dim=0)
                hour, day = move_forward_1h(hour, day)
    
    return workday_tensor, holiday_tensor
        

In [18]:
x_workday_tensor, x_holiday_tensor = split_work_holiday(x_tensor, day, hour)
y_workday_tensor, y_holiday_tensor = split_work_holiday(y_tensor, day, hour)

In [19]:
print(x_workday_tensor.shape)
print(y_workday_tensor.shape)
print(x_holiday_tensor.shape)
print(y_holiday_tensor.shape)

torch.Size([3073, 19, 128, 2])
torch.Size([3073, 1, 128, 2])
torch.Size([1242, 19, 128, 2])
torch.Size([1242, 1, 128, 2])


In [20]:
y_workday_tensor = y_workday_tensor.permute(0, 1, 3, 2)
y_holiday_tensor = y_holiday_tensor.permute(0, 1, 3, 2)
x_workday_tensor = x_workday_tensor.permute(0, 1, 3, 2)
x_holiday_tensor = x_holiday_tensor.permute(0, 1, 3, 2)

In [21]:
print(x_workday_tensor.shape)
print(y_workday_tensor.shape)
print(x_holiday_tensor.shape)
print(y_holiday_tensor.shape)
print(x_tensor.shape)
print(y_tensor.shape)
print(c.shape)
print(time_label.shape)

torch.Size([3073, 19, 2, 128])
torch.Size([3073, 1, 2, 128])
torch.Size([1242, 19, 2, 128])
torch.Size([1242, 1, 2, 128])
torch.Size([4315, 19, 128, 2])
torch.Size([4315, 1, 128, 2])
torch.Size([4315, 1, 128, 2])
torch.Size([4315])


In [22]:
workday_scaler = normalize_data(x_workday_tensor, 'Standard')
holiday_scaler = normalize_data(x_workday_tensor, 'Standard')
total_scalar = normalize_data(x_tensor, 'Standard')
x_workday_tensor = workday_scaler.transform(x_workday_tensor).to(torch.float)
x_holiday_tensor = holiday_scaler.transform(x_holiday_tensor).to(torch.float)
y_workday_tensor = workday_scaler.transform(y_workday_tensor).to(torch.float)
y_holiday_tensor = holiday_scaler.transform(y_holiday_tensor).to(torch.float)
x_total_tensor = total_scalar.transform(x_tensor).to(torch.float)
y_total_tensor = total_scalar.transform(y_tensor).to(torch.float)

In [23]:
workday_dataset = torch.utils.data.TensorDataset(x_workday_tensor, y_workday_tensor)
holiday_dataset = torch.utils.data.TensorDataset(x_holiday_tensor, y_holiday_tensor)
total_dataset = torch.utils.data.TensorDataset(x_total_tensor, y_total_tensor)

In [24]:
workday_total = len(workday_dataset)
holiday_total = len(holiday_dataset)
total_total = len(total_dataset)
workday_train_size = int(workday_total * 0.7)
holiday_train_size = int(holiday_total * 0.7)
total_train_size = int(total_total * 0.7)
workday_val_size = int(workday_total * 0.1)
holiday_val_size = int(holiday_total * 0.1)
total_val_size = int(total_total * 0.1)
workday_test_size = workday_total - workday_train_size - workday_val_size
holiday_test_size = holiday_total - holiday_train_size - holiday_val_size
total_test_size = total_total - total_train_size - total_val_size

In [43]:
workday_train, workday_val, workday_test = torch.utils.data.random_split(workday_dataset, [workday_train_size, workday_val_size, workday_test_size])
holiday_train, holiday_val, holiday_test = torch.utils.data.random_split(holiday_dataset, [holiday_train_size, holiday_val_size, holiday_test_size])
total_train, total_val, total_test = torch.utils.data.random_split(total_dataset, [total_train_size, total_val_size, total_test_size])

In [44]:
workday_test_dataloader = torch.utils.data.DataLoader(
    workday_test,
    batch_size=64,
    shuffle=False,
    drop_last=True
)

holiday_test_dataloader = torch.utils.data.DataLoader(
    holiday_test,
    batch_size=64,
    shuffle=False,
    drop_last=True
)

total_test_dataloader = torch.utils.data.DataLoader(
    total_test,
    batch_size=64,
    shuffle=False,
    drop_last=True
)

In [45]:
workday_set = {}
holiday_set = {}
total_set = {}

In [46]:
workday_set['dataloader'] = workday_test_dataloader
workday_set['scaler'] = workday_scaler
holiday_set['dataloader'] = holiday_test_dataloader
holiday_set['scaler'] = holiday_scaler
total_set['dataloader'] = total_test_dataloader
total_set['scaler'] = total_scalar

In [47]:
with open("workday_test_dataloader.pkl", "wb") as f:
    pickle.dump(workday_set, f)

with open("holiday_test_dataloader.pkl", "wb") as f:
    pickle.dump(holiday_set, f)

with open("total_test_dataloader.pkl", "wb") as f:
    pickle.dump(total_set, f)